In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: See the [Calendar](https://htmlpreview.github.io/?https://github.com/UBC-CS/cpsc330/blob/master/docs/calendar.html).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W1/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [6]:
low = ["footloose", "rigatoni", "sharpener", "thing"]
glove_wiki_vectors.most_similar(low[0])

[('falsettos', 0.5803160667419434),
 ('brigadoon', 0.5788702368736267),
 ('bedazzled', 0.5707419514656067),
 ('ladykillers', 0.5678316950798035),
 ('demented', 0.5490493178367615),
 ('flashdance', 0.5475085377693176),
 ('tootsie', 0.5429037809371948),
 ('miserables', 0.5370612144470215),
 ('caddyshack', 0.5360159873962402),
 ('grifters', 0.5344991683959961)]

In [7]:
glove_wiki_vectors.most_similar(low[1])

[('penne', 0.8219619393348694),
 ('fusilli', 0.8033047914505005),
 ('orecchiette', 0.765510082244873),
 ('tagliatelle', 0.7451848983764648),
 ('pappardelle', 0.7351698279380798),
 ('rotini', 0.7195174694061279),
 ('ziti', 0.7073922753334045),
 ('linguine', 0.6995640397071838),
 ('farfalle', 0.6981773972511292),
 ('fettuccine', 0.666634738445282)]

In [8]:
glove_wiki_vectors.most_similar(low[2])

[('sharpeners', 0.6974294185638428),
 ('mandoline', 0.5768365263938904),
 ('pencil', 0.5616449117660522),
 ('raclette', 0.5334018468856812),
 ('eraser', 0.5304221510887146),
 ('lampshade', 0.5290104746818542),
 ('chopstick', 0.527364194393158),
 ('juicer', 0.519385576248169),
 ('dropshot', 0.5183666944503784),
 ('scraper', 0.5163193345069885)]

In [9]:
glove_wiki_vectors.most_similar(low[3])

[('really', 0.9004601240158081),
 ('think', 0.8768858909606934),
 ('maybe', 0.875111997127533),
 ('things', 0.8741226196289062),
 ('know', 0.8708787560462952),
 ('you', 0.8658700585365295),
 ('else', 0.8604204058647156),
 ('kind', 0.8599962592124939),
 ('?', 0.8594360947608948),
 ('something', 0.8547258377075195)]

In [10]:
...

Ellipsis

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [11]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

In [12]:
glove_wiki_vectors.similarity("coast","shore")

0.70002705

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [13]:
for i in word_pairs:
    print(f"{i} : {glove_wiki_vectors.similarity(i[0],i[1])}")

('coast', 'shore') : 0.7000270485877991
('clothes', 'closet') : 0.5462760329246521
('old', 'new') : 0.6432487368583679
('smart', 'intelligent') : 0.7552731037139893
('dog', 'cat') : 0.8798074126243591
('tree', 'lawyer') : 0.07671947777271271


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [14]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [15]:
for i in test_words:
    if i in glove_wiki_vectors:
        print(True)
    else:
        print(False)

False
False
True
False
False
False
False
False
True
True


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [16]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [17]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [18]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.14283244

In [19]:
glove_wiki_vectors.similarity("white", "success")

0.3518238

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [20]:
glove_wiki_vectors.similarity("asian", "math")

0.16754797

In [21]:
glove_wiki_vectors.similarity("white", "math")

0.123169966

In [22]:
analogy("woman", "cooking", "man")

woman : cooking :: man : ?


,Analogy word,Score
0,barbecue,0.637068
1,meal,0.628370
2,ingredients,0.612181
3,cooked,0.611385
4,pasta,0.607415
5,cook,0.605678
6,dishes,0.603275
7,kitchen,0.602573
8,frying,0.593025
9,mix,0.591866


In [23]:
analogy("man", "driving", "woman")

man : driving :: woman : ?


,Analogy word,Score
0,car,0.635288
1,driver,0.627889
2,speeding,0.617300
3,vehicle,0.608771
4,drivers,0.599836
5,riding,0.591213
6,drunk,0.582134
7,truck,0.578620
8,driven,0.566886
9,cars,0.566099


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

_1) Our observations suggest that, despite being trained on a large and diverse dataset, these embeddings still contain some biases and stereotypes. For instance, we found that certain analogies perpetuated gender stereotypes, and some words had biased associations._ 
_For example, we observed that certain words related to women were more closely associated with domestic or nurturing roles, while words related to men were more closely associated with professional or leadership roles. These biases can perpetuate existing social inequalities and reinforce stereotypes._

_2) Using embeddings with biases can cause sub-optimal results and choices. Taking the example from above biases such as the ones we saw between success for aboriginal and white can affect the way in which AI writes thus cause these biases to be propogated further._

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [24]:
from sklearn.datasets import fetch_20newsgroups

In [25]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [26]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [27]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [30]:
def preprocess_text(text):
    text = text.replace("\n", " ")
    text = " ".join(["URL" if word.startswith(("http", "www")) else word for word in text.split()])
    text = " ".join(["EMAIL" if "@" in word else word for word in text.split()])
    text = " ".join(["NUM" if word.isdigit() else word for word in text.split()])

    doc = nlp(text)

    tokens = [
        token.lemma_  
        for token in doc
        if not token.is_stop  
        and not token.is_punct  
        and not token.is_space  
        and token.pos_ not in ["PRON", "DET"]  
    ]
    
    return " ".join(tokens)

In [31]:
df["text_pp"] = df["text"].apply(preprocess_text)

In [32]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,Actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late UPI Foreign Ministry spokesman Ferhat Ata...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,hi like subscribe Leadership Magazine wonder d...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

_During preprocessing, I removed new line characters to avoid inconsistencies during tokenization. I standardized links, emails, and numbers by converting them to "URL," "EMAIL," and "NUM" tokens, respectively, for consistent representation. Additionally, I removed stop words like "the" and "and" to reduce noise in the text. Punctuation and unnecessary spaces were eliminated as they do not carry semantic meaning. Finally, I removed pronouns (e.g., "he," "she") and shorter words like "a" to focus on more meaningful terms for analysis.._

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

_I experimented with different values for the number of topics (n_components), including 3, 4, 5, and 6. After evaluating the results, I selected 3 topics because this value provided the most coherent and interpretable document-topic associations._

Specifically:<br>
1)The document-topic distribution showed clear separations, where most documents were strongly associated with a single topic, making it easier to interpret the results.<br>
2)The topic-word distributions revealed meaningful clusters of words for each topic, which aligned well with the underlying themes in the dataset.<br>
3) Increasing the number of topics (e.g., 5 or 6) resulted in more fragmented and overlapping topics, reducing interpretability.

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(stop_words="english")
df_x = vec.fit_transform(df["text_pp"])
df_x

<4563x37166 sparse matrix of type '<class 'numpy.int64'>'
	with 284074 stored elements in Compressed Sparse Row format>

In [51]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 3
lda_model = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=42
)
document_topics = lda_model.fit_transform(df_x) 
document_topics

array([[0.04464709, 0.0072361 , 0.94811681],
       [0.13425622, 0.18912657, 0.6766172 ],
       [0.23751773, 0.29093339, 0.47154888],
       ...,
       [0.0071432 , 0.00761984, 0.98523696],
       [0.24546849, 0.7458937 , 0.00863781],
       [0.00112083, 0.00111462, 0.99776455]])

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

In [52]:
import mglearn

sorting = np.argsort(lda_model.components_, axis=1)[:, ::-1]
feature_names = np.array(vec.get_feature_names_out())

mglearn.tools.print_topics(
    topics=range(3),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       
--------      --------      --------      
num           god           people        
email         know          think         
file          say           num           
game          people        gun           
team          like          year          
program       think         know          
image         come          like          
play          time          right         
run           window        time          
entry         jesus         good          




<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [53]:
pd.DataFrame(document_topics[0:5])

,0,1,2
0,0.044647,0.007236,0.948117
1,0.134256,0.189127,0.676617
2,0.237518,0.290933,0.471549
3,0.015812,0.974577,0.009611
4,0.007866,0.008062,0.984072


_The model effectively assigns clear dominant topics for documents with strong thematic content (e.g., Documents 0, 3, and 4). However, for documents with mixed content (e.g., Documents 1 and 2), topic overlaps suggest the need for refinement, such as better preprocessing or revisiting the number of topics to improve separation_

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

_1) The goal of recommender systems is to make the best possible recommendation to a user given a set of features/data. Content-based filtering uses the content of the items instead of just their attributes to filter options and make recommendations. Instead of relying on extrinsic attributes which have been assigned, content based filtering matches items based on content and filters them to make recommendations._

_2) Recommender systems can prevent users from exploring by consistently recommending items of the same type/content. A user may never get introduced to a different genre of items due to recommender systems consistently and constantly recommending the same type of items. Recommender systems can also perpetuate biases as experienced in question 1. This can reinforce stereotypes as all the content that reaches a particular person can be filtered to only show items with that bias._

_3) Transfer learning in NLP is a technique where a pre-trained model is used as a starting point for a new task. The pre-trained model has already learned to recognize patterns in language from a large dataset, and this knowledge is transferred to the new task, allowing the model to adapt quickly with less training data._

_This technique reduces training time and data requirements, improves model performance, and leverages knowledge from large datasets. Pre-trained language models like BERT and Word2Vec are often used for transfer learning in NLP._

<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)